# Softmax_Zoo_Classifier
* softmax를 사용하여 zoo 데이터를 분류 
* 동물 데이터는 7개의 클래스로 구분한다. 

In [3]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(555)

#### 동물 종류를 분류할 모델 작성 

In [20]:
class Softmax_Zoo_Classifer(tf.keras.Model):
    
    # 생성자에서 W와 b에 대한 초기화 수행 
    def __init__(self, input_num, nb_classes):
        super(Softmax_Zoo_Classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((input_num, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        self.vars = [self.W, self.b]
        print(self.W.shape)
    
    #  로지스틱 함수
    def logit_fn(self, X):
        return tf.matmul(X, self.W) + self.b
    
    # 가설 함수 
    def hypothesis(self, X):
        return tf.nn.softmax(self.logit_fn(X))
    
    # 비용 함수 
    # - categorical_crossentropy 을 이용하여 비용 계산 
    def cost_fn(self, X, Y):
        logits = self.logit_fn(X)  # x에 대한 가설값 
        cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True)
        cost = tf.reduce_mean(cost_i)
        return cost
    
    # 기울기 함수 
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(X, Y)
            grads = tape.gradient(cost, self.vars)            # [W, b]
            return grads
        
    # 예측 함수 
    def prediction(self, X, Y):
        pred = tf.argmax(self.hypothesis(X), 1)                   # 행에서 가장 큰 숫자의 위치 반환 
        accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(Y, 1)), dtype=tf.float32))
        return accuracy
    
    # 학습 함수 
    def fit(self, X, Y, epochs=1000, verbose=100):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.vars))  # [W, b]
            if (i==0) | ((i+1)%verbose==0):
                acc = self.prediction(X, Y).numpy()
                loss = self.cost_fn(X, Y).numpy()
                print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

#### 사용할 데이터 입력 

In [21]:
xy = np.loadtxt('../datas/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, :-1]      # 16종류의 데이터 읽어 오기 
y_data = xy[:, [-1]]     # 마지막 데이터 읽어오기(정답)
print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


#### softmax 처리를 하기위해 y_data를 one_hot으로 변경 

In [26]:
nb_classes = 7  # 동물에 대한 클래스 분류 0 ~ 6
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)
print(Y_one_hot.shape)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print(Y_one_hot.shape)

(101, 1, 7)
(101, 7)


In [27]:
model = Softmax_Zoo_Classifer(16, nb_classes)

(16, 7)


In [29]:
model.fit(x_data, Y_one_hot)

Steps: 1 Loss: 0.11865100264549255, Acc: 0.9900990128517151
Steps: 100 Loss: 0.10821477323770523, Acc: 0.9900990128517151
Steps: 200 Loss: 0.09925895184278488, Acc: 0.9900990128517151
Steps: 300 Loss: 0.09156828373670578, Acc: 0.9900990128517151
Steps: 400 Loss: 0.08490054309368134, Acc: 0.9900990128517151
Steps: 500 Loss: 0.07907363027334213, Acc: 1.0
Steps: 600 Loss: 0.07394677400588989, Acc: 1.0
Steps: 700 Loss: 0.06940881907939911, Acc: 1.0
Steps: 800 Loss: 0.06537053734064102, Acc: 1.0
Steps: 900 Loss: 0.06175912171602249, Acc: 1.0
Steps: 1000 Loss: 0.058514486998319626, Acc: 1.0
